In [1]:
# Import needed packages
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
import csv
import sys
import pandas_datareader as pdr
import datetime


In [14]:
startdate = datetime.datetime(2022,3,28)
enddate = datetime.datetime(2022,4,1)
st = pdr.get_data_yahoo('MMM',start=startdate,end=enddate)
st

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-03-28,150.130005,147.190002,149.929993,149.470001,2679900,149.470001
2022-03-29,152.919998,150.039993,151.000000,152.080002,2693200,152.080002
2022-03-30,153.259995,150.770004,151.330002,151.429993,2395100,151.429993
2022-03-31,151.250000,148.800003,150.449997,148.880005,3193800,148.880005
2022-04-01,149.860001,146.850006,149.630005,149.690002,2196400,149.690002


In [26]:
# Get Twitter data and do sentiment analysis
# Authenticate credentials

#Twitter credentials for the app
consumer_key = 'lLoOz2cbplDwX3vTzZFv2Hyqz'
consumer_secret = 'Yp5YFPvY3ISI2KUVQC6HAqYjDCUWw5na3IFrCvTVoTuvF9QddN'
access_key= '3028339548-i1TWGaqfItqlybCtkJZUQ7km95pylMZl8zPQBLb'
access_secret = 'llC8DaUitlD5wMl0FMqSxR3L78q0OaF56oazKJF5Hqnnr'

#Pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key,access_secret)
# api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify=True)
api = tweepy.API(auth)

query = "$"+"MMM" + " -filter:retweets -filter:replies"
tweets = tweepy.Cursor(api.search_tweets ,q=query,lang='en', since = startdate, until = enddate, tweet_mode='extended').items(5)
print(tweets)
for tweet in tweets:
    status=tweet._json
    created_at = status['created_at']
    tweet_id = status['id_str']
    tweet_text = status['full_text']
    tweet_text = re.sub(r'\W', ' ',tweet_text)
    print(created_at)

Unexpected parameter: since


Thu Mar 31 22:51:05 +0000 2022
Thu Mar 31 22:30:02 +0000 2022
Thu Mar 31 20:28:37 +0000 2022
Thu Mar 31 17:33:06 +0000 2022
Tue Mar 29 23:03:05 +0000 2022


In [22]:
# Obtain stock tickers

data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S&P_500_companies')
#print(data)
table = data[0]
#table.head()
tickers = table['Symbol'].tolist()
tickers1 = tickers[:99]
tickers2 = tickers[99:199]
tickers3 = tickers[199:299]
tickers4 = tickers[299:399]
tickers5 = tickers[399:499]
tickers6 = tickers[499:]
print(tickers2)

['CDAY', 'CERN', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DISCA', 'DISCK', 'DISH', 'DIS', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE', 'DUK', 'DRE', 'DD', 'DXC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'EMR', 'ENPH', 'ETR', 'EOG', 'EPAM', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY', 'RE', 'EVRG', 'ES', 'EXC', 'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FDS', 'FAST', 'FRT', 'FDX', 'FITB', 'FRC']


In [4]:
# Get stock prices and returns
startdate = datetime.datetime(2022,3,21)
enddate = datetime.datetime(2022,3,25)

file = open("stocks.csv",'w',newline='')
print(len(tickers))
count = 0
for ticker in tickers:
    count +=1
    print(count)
    try:
        st = pdr.get_data_yahoo(ticker,start=startdate,end=enddate)
        st['pct_change'] = st['Adj Close'].pct_change(4)
        #print(ticker + str(st['Adj Close']) + str(st['pct_change']))
        stockdata = csv.writer(file)
        stockdata.writerow([ticker,st['pct_change'].iloc[-1]])
    except:
        pass



505
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


In [27]:
# Get Twitter data and do sentiment analysis
# Authenticate credentials

#Twitter credentials for the app
consumer_key = 'lLoOz2cbplDwX3vTzZFv2Hyqz'
consumer_secret = 'Yp5YFPvY3ISI2KUVQC6HAqYjDCUWw5na3IFrCvTVoTuvF9QddN'
access_key= '3028339548-i1TWGaqfItqlybCtkJZUQ7km95pylMZl8zPQBLb'
access_secret = 'llC8DaUitlD5wMl0FMqSxR3L78q0OaF56oazKJF5Hqnnr'

#Pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key,access_secret)
# api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify=True)
api = tweepy.API(auth)

file = open("searchTweets.csv","w",newline='')
alltweets = csv.writer(file)
print(len(tickers))
# count = 0
# for ticker in tickers1:
#     count += 1
#     print(count)
#     query = "$"+ticker + " -filter:retweets -filter:replies"
#     tweets = tweepy.Cursor(api.search_tweets ,q=query,lang='en', until = enddate, tweet_mode='extended').items(10)
#     for tweet in tweets:
#         status=tweet._json
#         created_at = status['created_at']
#         tweet_id = status['id_str']
#         tweet_text = status['full_text']
#         tweet_text = re.sub(r'\W', ' ',tweet_text)
#         print(tweet_text)
#         try: 
#             alltweets.writerow([ticker,created_at,tweet_id, tweet_text])
#         except UnicodeEncodeError:
#             pass
# count = 100
# for ticker in tickers2:
#     count += 1
#     print(count)
#     query = "$"+ticker + " -filter:retweets -filter:replies"
#     tweets = tweepy.Cursor(api.search_tweets,q=query,lang='en', until = enddate, tweet_mode='extended').items(10)
#     for tweet in tweets:
#         status=tweet._json
#         created_at = status['created_at']
#         tweet_id = status['id_str']
#         tweet_text = status['full_text']
#         tweet_text = re.sub(r'\W', ' ',tweet_text)
#         print(tweet_text)
#         try: 
#             alltweets.writerow([ticker,created_at,tweet_id, tweet_text])
#         except UnicodeEncodeError:
#             pass

        
count = 200
for ticker in tickers3:
    count += 1
    print(count)
    query = "$"+ticker + " -filter:retweets -filter:replies"
    tweets = tweepy.Cursor(api.search_tweets,q=query,lang='en', until = enddate, tweet_mode='extended').items(10)
    for tweet in tweets:
        status=tweet._json
        created_at = status['created_at']
        tweet_id = status['id_str']
        tweet_text = status['full_text']
        tweet_text = re.sub(r'\W', ' ',tweet_text)
        print(tweet_text)
        try: 
            alltweets.writerow([ticker,created_at,tweet_id, tweet_text])
        except UnicodeEncodeError:
            pass


505
101
 CDAY Max Pain is 65 00 for maturity 04 14 2022   maxpain  options https   t co X3tjSzJyab https   t co FNC3SE0SNY
102
103
 CF Strike  110 00 Expiration  2022 04 14 Call  CF220414C110 Price  2 08 Size  1263 Time  2022 03 30 10 16 21AM Premium   262 704 00
 CF high OI range is 94 00 to 118 00 for option expiration 04 01 2022 PutCallRatio 0 67  maxpain  options https   t co F42H33G2P6 https   t co mWHTzQSeNj
DailyDown5  ADM  87 58  5 17  0 02  11 00  35 53  58 94  CF  99 16  5 67   0 84  24 73  38 24  57 60  CHWY  44 10  9 63   2 91  4 58   20 91  49 77  GME  179 56  5 29  45 82  51 43  16 60  72 70  NTIC  11 83  6 92   6 85   11 05   14 28  36 68
DailyDown5  ADM  86 59  6 24   1 11  9 75  34 00  56 37  CF  97 72  7 04   2 28  22 92  36 23  55 75  CLF  31 08  6 02  9 48  44 49  54 01  67 99  DE  410 00  5 23   4 10  18 16  18 16  55 31  FMC  129 15  5 00   1 06  9 48  20 14  56 80
DailyDown5  AA  87 23  5 66   2 95  12 04  47 07  56 03  ADM  86 98  5 81   0 66  10 24  34 60  57 3

Citrix Systems Inc    CTXS    QUOTE  CTXS was DOWN   on Tue Mar 29 Closed at 101 47 USD for a 0 00  loss from previous close          ANALYSIS Citrix Systems Inc should be nationalized      
118
   CLX price target update    Target Lowered by JPMorgan Chase  amp  Co  From  137 00 to  123 00  Day quote change   139 33   0 24   Target upside   11 72   Published  March 30  2022 https   t co S9i4wREQAX
   CLX price target update    Target Lowered by The Goldman Sachs Group From  145 00 to  127 00  Day quote change   125  0 75   Target upside  1 6   Published  March 29  2022 https   t co S9i4wREQAX
   CLX price target update    Target Lowered by The Goldman Sachs Group From  145 00 to  127 00  Day quote change   140 4  1 37   Target upside   9 54   Published  March 29  2022 https   t co S9i4wRWrZx
119
Nice print for  CME Size  150000 Price  240 949 Amount   36 142 350 00 Time  1502 See more  https   t co 2z9CUvBlnm Join https   t co JWvlYbdLod to get REAL TIME prints  Runners    https   t c

   ED price target update    Target Raised by The Goldman Sachs Group From  65 00 to  83 00  Day quote change   94 58   0 05   Target upside   12 24   Published  March 31  2022 https   t co 71qxtquhBj
Consolidated Edison  ED PT Raised to  83 00 at The Goldman Sachs Group https   t co B1i2ulzBtU  markets
Consolidated Edison  ED PT Raised to  83 00 at The Goldman Sachs Group https   t co xdL9TXkL3h  stocks
129
 STZ Strike  230 00 Expiration  2022 04 08 Call  STZ220408C230 Price  5 70 Size  1000 Time  2022 03 30 9 36 21AM Premium   570 000 00
 STZ Strike  240 00 Expiration  2022 04 08 Call  STZ220408C240 Price  1 85 Size  1000 Time  2022 03 30 9 36 22AM Premium   185 000 00
   STZ price target update    Target Lowered by Credit Suisse Group From  285 00 to  278 00  Day quote change   231 01  1 04   Target upside  20 34   Published  March 29  2022 https   t co jK5NAWfWre
Constellation Brands Inc    STZ    QUOTE  STZ was DOWN   on Tue Mar 29 Closed at 231 75 USD for a 0 00  loss from previo

143
144
145
 DE  480 00 May 20 2022 CALL     Underlying   417 62   Difference  12 99  Daily   Volume   689 028 Volume  2 571 OI  145 IV  30 23  Bid Ask   2 59  2 77
DailyDown5  ADM  86 59  6 24   1 11  9 75  34 00  56 37  CF  97 72  7 04   2 28  22 92  36 23  55 75  CLF  31 08  6 02  9 48  44 49  54 01  67 99  DE  410 00  5 23   4 10  18 16  18 16  55 31  FMC  129 15  5 00   1 06  9 48  20 14  56 80
146
Nice print for  DAL Size  111025 Price  39 4 Amount   4 374 385 00 Time  930 See more  https   t co UT7YjXUUxV Join https   t co JWvlYbdLod to get REAL TIME prints  Runners    https   t co VC1LdPHaJR Losers    https   t co 9MWV4THfhQ Gappers    https   t co bHQKF1W40n
 DAL  44 00 May 20 2022 CALL     Underlying   39 41   Difference  10 42  Daily   Volume   100 956 Volume  1 128 OI  226 IV  40 67  Bid Ask   0 88  0 91
 DAL Strike  45 00 Expiration  2022 06 17 Call  DAL220617C45 Price  1 12 Size  2439 Time  2022 03 30 10 05 14AM Premium   273 168 00
 DAL Strike  45 00 Expiration  2022 06 

160
161
 DOV  15s  delayed   Issued Press Release on March 30  16 15 00  SWEP Enhances All Stainless Range of Brazed Plate Heat Exchangers with B222 Large Size Model https   t co SSQ2XpNa4A
 DOV Max Pain 160 00  Maturity 04 14 2022   maxpain  options https   t co 1ve3pTm8w1 https   t co poDbrnfNde
 GGG short ratio on 2019 05 31 is 4 00 https   t co 8ITqOVTpWQ  IEX 3 83  GRC 1 89  DOV 4 40  FLS 4 52
162
Stock   DOW        REALTIME TREND     BUY BUY   95 00   SELL   5 00   Try now the Best Pivot Point Calculator   https   t co 7dVo6k7x4v  tradingview  stocks  stockstowatch
 DOW Strike  65 00 Expiration  2022 04 14 Call  DOW220414C65 Price  0 77 Size  2000 Time  2022 03 29 12 15 01PM Premium   154 000 00
163
 DTE  15s  delayed   Issued Press Release on March 29  09 14 00  DTE  Lyft partner to make electric vehicles more accessible https   t co 8lj3jl4hZ8
164
 DUK  15s  delayed   Issued Press Release on March 31  16 06 00  Piedmont Natural Gas  Duke Energy remind communities to call 811 be

183
184
   EL price target update    Target Lowered by JPMorgan Chase  amp  Co  From  338 00 to  317 00  Day quote change   277 92   2 43   Target upside  14 06   Published  March 30  2022 https   t co nUuMPQQsOA
185
DailyDown5  DELL  50 30  7 40   6 00  0 84   11 13  39 47  DIDI  2 53  13 95   33 42   37 99   52 26  37 81  DWAC  63 98  6 56   11 79   32 72  20 81  36 14  EH  12 22  5 64   7 42   12 59   17 93  46 75  ETSY  124 79  7 26   11 47   20 78   45 67  41 45
DailyDown5  ETSY  124 80  7 25   11 46   20 78   45 66  41 45  EXFY  17 40  6 75   3 81   20 62   54 82  39 72  EXPI  21 17  5 32   10 45   20 65   41 00  39 12  FUBO  6 65  6 76   9 06   21 51   60 45  38 60  GFS  63 00  6 00   17 34  8 21   1 56  46 61
DailyDown5  DELL  50 28  7 44   6 04  0 80   11 17  39 42  DIDI  2 51  14 63   33 95   38 48   52 64  37 62  DWAC  64 30  6 09   11 35   32 39  21 41  36 46  EH  12 22  5 64   7 42   12 59   17 93  46 75  ETSY  125 00  7 10   11 32   20 65   45 57  41 54
 ETSY just sold th

198
Stock   FDX        REALTIME TREND     BUY BUY   95 00   SELL   5 00   Try now the Best Pivot Point Calculator   https   t co WLEIyDRkLR  tradingview  stocks  stockstowatch
Stock   FDX        REALTIME TREND     BUY BUY   95 00   SELL   5 00   Try now the Best Pivot Point Calculator   https   t co m9PXii6wDX  tradingview  stocks  stockstowatch
Nice print for  FDX Size  160324 Price  235 25 Amount   37 716 221 00 Time  1830 See more  https   t co QJbK2jB66G Join https   t co JWvlYbdLod to get REAL TIME prints  Runners    https   t co VC1LdPHaJR Losers    https   t co 9MWV4THfhQ Gappers    https   t co bHQKF1W40n
A I dvisor made a bullish call on  FDX  netting in a 13 00  gain over 2 weeks  FedEx https   t co 5HINCUU4Cd https   t co p2GHhaiRDo
Stock   FDX        REALTIME TREND     BUY BUY   95 00   SELL   5 00   Try now the Best Pivot Point Calculator   https   t co eIpP1MOcWZ  tradingview  stocks  stockstowatch
Today s Market Heatmap   March 30   TSLA  0 76   AAPL  0 01   C  0 45   AM

In [ ]:
# Sentiment analysis

file = open("searchTweets.csv","r")
alltweets = csv.reader(file)

sentiment_file = open("sentimenttweets.csv","w",newline='')
sntTweets=csv.writer(sentiment_file)

for row in alltweets:
    blob = TextBlob(row[3])
    if blob.sentiment.polarity>0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"positive"])
    elif blob.sentiment.polarity<0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"negative"])     
    elif blob.sentiment.polarity==0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"neutral"])    
        
file.close()
sentiment_file.close()